In [ ]:
# DATA SET CANCER DE MAMA https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29

### RECORDATORIO ayuda de panda recordset
df = pd.read_csv('sp500_ohlc.csv', index_col = 'Date', parse_dates=True)
print df.head()
df2 = df['Open']
print df2.head()
df3 = df[['Close','High']]
print df3.head()
df3.rename(columns={'Close': 'CLOSE!!'}, inplace=True)
print df3.head()
df4 = df3[(df3['CLOSE!!'] > 1400)]
print df4

### panda recordatorio OPERACIONES DE COLUMNAS
import pandas as pd
from pandas import DataFrame
df = pd.read_csv('sp500_ohlc.csv', index_col = 'Date', parse_dates=True)
#notice what i did, since it is an object
df['H-L'] = df.High - df.Low

print df.head()
df['100MA'] = pd.rolling_mean(df['Close'], 100)

#### must do a slice, since there will be no value for 100ma until 100 points
print df[200:210]

df['Difference'] = df['Close'].diff()
print df.head()

#### mas sobre ILOC
df = pd.DataFrame({'A':['a', 'b', 'c'], 'B':[54, 67, 89]}, index=[100, 200, 300])

df

                        A   B
                100     a   54
                200     b   67
                300     c   89
In [19]:    
df.loc[100]

Out[19]:
A     a
B    54
Name: 100, dtype: object

In [20]:    
df.iloc[0]

Out[20]:
A     a
B    54
Name: 100, dtype: object

In [24]:    
df2 = df.set_index([df.index,'A'])
df2

Out[24]:
        B
    A   
100 a   54
200 b   67
300 c   89

In [25]:    
df2.ix[100, 'a']

Out[25]:    
B    54
Name: (100, a), dtype: int64


## Aunque esta para python 3.4 originalmente lo estoy ejecutando en 2.7 de forma que documentare las funciones que sean incompatibles

#### atencion, para aprendizaje, creo un archivo txt de origenes de datos llamado breast-cancer-wisconsin.data.txt que no es el original (breast-cancer-wisconsin.data-TOTAL.txt), sino una copia con TRES CAMPOS + 1 LABEL . 

In [1]:
import numpy as np
# from sklearn import preprocessing, cross_validation, neighbors
import pandas as pd

df = pd.read_csv('breast-cancer-wisconsin.data_total.txt')
df.replace('?',-99999, inplace=True)
df.drop(['id'], 1, inplace=True)



In [2]:
df.head()

,Clump_Thickness,Unifom_Cell_Size,Uniform_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [ ]:
print(df.columns)

In [ ]:
print(df.values)

In [ ]:
print(df.describe)

In [ ]:
df['Class']

In [ ]:
df[0:2]

In [116]:
df.iloc[2]

Clump_Thickness                3
Unifom_Cell_Size               1
Uniform_Cell_Shape             1
Marginal_Adhesion              1
Single_Epithelial_Cell_Size    2
Bare_Nuclei                    2
Bland_Chromatin                3
Normal_Nucleoli                1
Mitoses                        1
Class                          2
Name: 2, dtype: object

In [ ]:
# condiciones 
df[df.Class > 3]

In [ ]:
df.head()

In [3]:
# CLASIFICADOR montamos los arrays de NUMPY

X = np.array(df.drop(['Class'], 1))
y = np.array(df['Class'])

In [4]:
# a partir de x e y, planteamiento original de clasificador svm 

from sklearn import preprocessing, cross_validation, neighbors, svm

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

# utilizamos el support vector classifier  
clf = svm.SVC()

clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
print(confidence)

0.978571428571


# VARIOS CLASIFICADORES CON CROSSFOLDER MEDIANTE LA FUNCION SIGUIENTE:  run_cv(X,y,clf_class)

In [5]:
# -------------------------  VARIOS CLASIFICADORES ----------------------------

from sklearn.cross_validation import KFold

#  definimos la funcion de cross validation
def run_cv(X,y,clf_class,**kwargs):
    # Construimos el objeto kfolds 
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
    
    # tenemos  que iterar sobre todos los kfolds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        
        
        # Iniciamos el clasificador con los keywords arguments...
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [24]:


from sklearn import preprocessing, cross_validation, neighbors, svm
from sklearn.metrics import *
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# AÑADIMOS  varios clasificadores al SVC Original 
from sklearn.svm import SVC
# estos nuevos
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LOG_REG


X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

# utilizamos el support vector classifier  
clf = svm.SVC()
clf2 = neighbors.KNeighborsClassifier()
clf3 = LogisticRegression(C=0.001, penalty='l2')


#clf3 = ensemble.RandomForestClassifier()
#clf_RF = ensemble.RandomForestClassifier()   esto da error

clf.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

# RF.fit(X_train, y_train)  me da error al aplicar la funcion así

# ----- Lista de Predictiones -----------------
predictions = clf.predict(X_test)
predictions2 = clf2.predict(X_test)
predictions3 = clf3.predict(X_test)


# ------- Printing ---------------
print "--------  Confianzas de los calsificadores: "
confidence_SVC = clf.score(X_test, y_test)
print "Confianza de SVC = ", confidence_SVC
print "MATRIZ DE CONFUSION de SVC \n", confusion_matrix(y,run_cv(X,y,SVC)), "\n"

confidence_KNN = clf2.score(X_test, y_test)
print "Confianza de KNN = ", confidence_KNN
print "MATRIZ DE CONFUSION de KNN \n", confusion_matrix(y,run_cv(X,y,KNN)), "\n"

confidence_LOG_REG = clf3.score(X_test, y_test)
print "Confianza de  Log. Regression = ", confidence_LOG_REG
print "MATRIZ DE CONFUSION de Log. Regression \n", confusion_matrix(y,run_cv(X,y,LOG_REG)), "\n"

# CON RANDOM FOREST, usamos la funcion anterior de croos folder
def accurade(y_true,y_pred):
    # NumPy interprets True and False as 1. and 0.
    return np.mean(y_true == y_pred)
#print "%.3f" % accuracy(y, run_cv(X,y,RF))
print "Confianza de Random forest:", accuracy(y_test, run_cv(X_test,y_test,RF))
#print "Random forest:", accuracy(y, run_cv(X,y,RF))
print "MATRIZ DE CONFUSION de RANDOM FOREST \n", confusion_matrix(y,run_cv(X,y,RF)), "\n"




--------  Confianzas de los calsificadores: 
Confianza de SVC =  0.971428571429
MATRIZ DE CONFUSION de SVC 
[[431  27]
 [  3 238]] 

Confianza de KNN =  0.978571428571
MATRIZ DE CONFUSION de KNN 
[[448  10]
 [ 10 231]] 

Confianza de  Log. Regression =  0.842857142857
MATRIZ DE CONFUSION de Log. Regression 
[[445  13]
 [ 15 226]] 

Confianza de Random forest: 0.957142857143
MATRIZ DE CONFUSION de RANDOM FOREST 
[[444  14]
 [ 17 224]] 




### NOTA En lo ultimo tambien prodiamos usar tal como hicimos en el del churn.. 
print "Support vector machines:"
print "%.3f" % accuracy(y, run_cv(X,y,SVC))
print "Random forest:"
print "%.3f" % accuracy(y, run_cv(X,y,RF))
print "K-nearest-neighbors:"
print "%.3f" % accuracy(y, run_cv(X,y,KNN))


####  ---       predicciones que no seé porque no funciona (???) -------------------------------
print "Predicciones para el SVM ----"
print 'Precision: ', precision_score(y_test, predictions)
print 'Recall: ', recall_score(y_test, predictions)
print 'Accuracy: ', accuracy_score(y_test, predictions)

print " Predicciones para el KNN ----"
print 'Precision: ', precision_score(y_test, predictions2)
print 'Recall: ', recall_score(y_test, predictions2)
print 'Accuracy: ', accuracy_score(y_test, predictions2)

###  etc...




### Calculamos los limites de los SLIDES de BOKEH para que estén derentro del margen de los valores del data set

In [133]:
# RECORREMOS AHORA TODO EL DATAFRAME DE FORMA QUE SEA APLICABLE A CUALQUIERA DE LOS DATA FRAMES DE LOS CASOS... 

lista_minimos_col = []
lista_maximos_col = []

# recorremos las columnas del DATA FRAME
for i in range (df.shape[1]): 
    #print df_sencillo.iloc[1][i:1]
    #print df_sencillo.values[1][i]
    # añadimos el MIN de esa columna del dataframe
    lista_minimos_col.append(df.ix[df[df.columns[i]].idxmin()][df.columns[i]])
    lista_maximos_col.append(df.ix[df[df.columns[i]].idxmax()][df.columns[i]])

print lista_minimos_col
print lista_maximos_col

[1, 1, 1, 1, 1, -99999, 1, 1, 1, 2]
[10, 10, 10, 10, 10, '9', 10, 10, 10, 4]


In [ ]:
# RECORREMOS AHORA TODO EL DATAFRAME DE FORMA QUE SEA APLICABLE A CUALQUIERA DE LOS DATA FRAMES DE LOS CASOS... 
import pandas 
df_sencillo = pandas.DataFrame(np.random.randn(5,3),columns=['A','B','C']) 
print df_sencillo

lista_minimos_col = []
lista_maximos_col = []

print "============================================"

# recorremos las columnas del DATA FRAME
for i in range (df_sencillo.shape[1]): 
    #print df_sencillo.iloc[1][i:1]
    #print df_sencillo.values[1][i]
    # añadimos el MIN de esa columna del dataframe
    lista_minimos_col.append(df_sencillo.ix[df_sencillo[df_sencillo.columns[i]].idxmin()][df_sencillo.columns[i]])
    lista_maximos_col.append(df_sencillo.ix[df_sencillo[df_sencillo.columns[i]].idxmax()][df_sencillo.columns[i]])

print lista_minimos_col
print lista_maximos_col

# Ahora calculamos los puntos cercanos a un punto dado para calcular donde están mas proximas los puntos de categoria diferente, (cancerosas). Esto nos ayudará a seleccionar las direcciones más relevantes a la hora de estudiar la evolución del punto en cuestión.
##UTLIZAMOS UN EJEMPLO SENCILLO DE DATAFRAME EN SU LUGAR

In [ ]:
# Para que funcionen bien los sliders dentro de los valores del Data Set, calculamos los VALORES MAXIMOS Y MINIMOS
# DE CADA COLUMNA DEEN EL MISMO
import pandas 
df_sencillo = pandas.DataFrame(np.random.randn(5,3),columns=['A','B','C'])  
print df_sencillo

print "esto nos da el indice del MAXIMO valor de la columna"
print df_sencillo.idxmax()
print "y esto el valor en las tres columnas A, B y C  "
registro_max= df_sencillo.ix[df_sencillo['A'].idxmax()]
print registro_max
print registro_max[0]

print "esto nos da el indice MINIMO"
print df_sencillo.idxmin()
print "y esto el valor en las tres columnas A, B y C  "
registro_min= df_sencillo.ix[df_sencillo['A'].idxmin()]
print registro_min
print "\n ", registro_min[0]

print "============================================"
print "AHORA generamos un valor random entre los EXTREMOS de cada columna -----"
import random
min_A= df_sencillo.ix[df_sencillo['A'].idxmin()]
max_A= df_sencillo.ix[df_sencillo['A'].idxmax()]

# Como A = df_sencillo.columns[0]   , esto se puede escribir como 
min_A= df_sencillo.ix[df_sencillo[df_sencillo.columns[0]].idxmin()][df_sencillo.columns[0]]
max_A= df_sencillo.ix[df_sencillo[df_sencillo.columns[0]].idxmax()][df_sencillo.columns[0]]
print "el minimo es = ", min_A 
print "el maximo es = ", max_A 

                  
muestra_random_emtre_limites = random.uniform(min_A,max_A)
print "============================================"

print "el numero random entre ", min_A, " y ", max_A,  "en la primera coluna es : ", muestra_random_emtre_limites 

### RECORDATORIO DE SELECCION DE CELDAS Y COLUMNAS
import pandas 
df_sencillo = pandas.DataFrame(np.random.randn(2,3),columns=['A','B','C']) 
print df_sencillo
### asi seleccionamos el NOMBRE de la ultima columna
print df_sencillo.columns[-1:]
### su valor es 
#print df_sencillo.columns[df_sencillo.columns[-1:]]
print df_sencillo.iloc[1][-1:]

### valores del dataframe
print df_sencillo.values[1][1]

### asi obtenemos el numero de columnas. el de filas es df_sencillo.shape[0]
#print df_sencillo.shape[1]

### RECORREMOS AHORA TODO EL DATAFRAME DE FORMA QUE SEA APLICABLE A CUALQUIERA DE LOS DATA FRAMES DE LOS CASOS... 
import pandas 
df_sencillo = pandas.DataFrame(np.random.randn(5,3),columns=['A','B','C']) 
print df_sencillo

lista_minimos_col = []
lista_maximos_col = []

print "============================================"

#### recorremos las columnas del DATA FRAME
for i in range (df_sencillo.shape[1]): 
    #print df_sencillo.iloc[1][i:1]
    #print df_sencillo.values[1][i]
    ####  añadimos el MIN de esa columna del dataframe
    lista_minimos_col.append(df_sencillo.ix[df_sencillo[df_sencillo.columns[i]].idxmin()][df_sencillo.columns[i]])
    lista_maximos_col.append(df_sencillo.ix[df_sencillo[df_sencillo.columns[i]].idxmax()][df_sencillo.columns[i]])

print lista_minimos_col
print lista_maximos_col




# PREDICCIONES CON INPUTS

In [ ]:
# vamos a hacer predicciones :

example_measures = np.array([[10,5,5,3,6,7,7,10,1]])
example_measures = example_measures.reshape(len(example_measures), -1)
prediction = clf.predict(example_measures)
print(prediction)

In [50]:
print(df.values)

[[5L 1L 1L ..., 1L 1L 2L]
 [5L 4L 4L ..., 2L 1L 2L]
 [3L 1L 1L ..., 1L 1L 2L]
 ..., 
 [5L 10L 10L ..., 10L 2L 4L]
 [4L 8L 6L ..., 6L 1L 4L]
 [4L 8L 8L ..., 4L 1L 4L]]


# 2 PARTE: gráficas-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
# condiciones que definen nuevos datasets
df_tumoral = df[df.Class > 3]
df_NO_tumoral = df[df.Class < 3]

df_tumoral.head()

In [ ]:
# graficas scattered 
import matplotlib.pyplot as plt
#scattercols = ['price','accommodates', 'number_of_reviews', 'reviews_per_month', 'beds', 'availability_30', 'review_scores_rating']
scattercols = df.columns
axs = pd.scatter_matrix(df[scattercols],
                        figsize=(12, 12), c='red')

plt.figure(figsize=(1920,1080))
plt.show()

In [ ]:
# definimos una funcion distancia de un registro del data se

In [ ]:
df_NO_tumoral.head()

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')

x = df_tumoral.Clump_Thickness
y = df_tumoral.Uniform_Cell_Shape
z = df_tumoral.Unifom_Cell_Size

x2 = df_NO_tumoral.Clump_Thickness
y2 = df_NO_tumoral.Uniform_Cell_Shape
z2 = df_NO_tumoral.Unifom_Cell_Size


ax1.scatter(x, y, z, c='g', marker='o')
ax1.scatter(x2, y2, z2, c ='r', marker='o')

ax1.set_xlabel('Espesor')
ax1.set_ylabel('Forma Celular')
ax1.set_zlabel(u'Tamaño Celular')

plt.show()

In [ ]:
#imprimimos las 3 primeras variables de prueba

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')



x = df_tumoral.eval(df_tumoral.columns[1])
y =  df_tumoral.eval(df_tumoral.columns[2])
z =  df_tumoral.eval(df_tumoral.columns[3])

x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[1])
y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[2])
z2 = df_NO_tumoral.eval(df_NO_tumoral.columns[3])


ax1.scatter(x, y, z, c='g', marker='o')
ax1.scatter(x2, y2, z2, c ='r', marker='o')

ax1.set_xlabel('Espesor')
ax1.set_ylabel('Forma Celular')
ax1.set_zlabel(u'Tamaño Celular')

plt.show()

In [ ]:
print(df.columns)

In [ ]:
print(df.columns[9])
print(len(df.columns))
      

In [ ]:
# funcion GENERICA QUE DIBUJA UNA COMBINACION particulr de las variable

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

def dibuja_grafica(i,j,k):

    style.use('ggplot')

    fig = plt.figure()
    ax1 = fig.add_subplot(111, projection='3d')


    x = df_tumoral.eval(df_tumoral.columns[i])
    y =  df_tumoral.eval(df_tumoral.columns[j])
    z =  df_tumoral.eval(df_tumoral.columns[k])

    x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[i])
    y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[j])
    z2 = df_NO_tumoral.eval(df_NO_tumoral.columns[k])

    ax1.scatter(x, y, z, c='g', label='TUMORALES', marker='o')
    ax1.scatter(x2, y2, z2, c ='r', label='SANAS', marker='o')
    
    #plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
    #plt.scatter(x2,y2, label='SANAS', color='r', s=25, marker="o")
 
    ax1.set_xlabel(df_tumoral.columns[i])
    ax1.set_ylabel(df_tumoral.columns[j])
    ax1.set_zlabel(df_tumoral.columns[k])
    
    plt.title('Breast Cancer Dataset Analysis               ')
    plt.legend()

    plt.show()  
    #plt.savefig('graf_'+df_tumoral.columns[i]+df_tumoral.columns[j]+df_tumoral.columns[k]+'.png')

dibuja_grafica(1,2,4)
    
    

    

In [ ]:
df.columns[-1]

In [ ]:
# representacion de todas las GRAFICAS combinacion total de variables

for counter1 in range(0,len(df.columns)-4):
    for counter2 in range(counter1 + 1,len(df.columns)):
        for counter3 in range(counter2 + 1,len(df.columns)):
            try:
                dibuja_grafica(counter1,counter2,counter3)
                #print counter1,",", counter2,",", counter3
            except Exception:
                pass

In [ ]:
# recordatorio de graficas en 2D

import matplotlib.pyplot as plt

x = [1,2,3,4,5,6,7,8]
y = [5,2,4,2,1,4,5,2]

x2 = [2,3,3,7,5,2,5]
y2 = [3,1,0,2,4,5,2]


#style.use('ggplot')

fig = plt.figure() 

# AÑADIMOS UN SUBPLOT sobre fig
ax1 = fig.add_subplot(111)

ax1.scatter(x, y, c='g', marker='o')
ax1.scatter(x2, y2, c ='r', marker='o')

plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
plt.scatter(x2,y2, label='NORMALES', color='r', s=25, marker="o")

plt.xlabel('x')
plt.ylabel('y')
plt.title('Breast Cancer Analysis')
plt.legend()
plt.show()

In [ ]:
# GRAFICA EN 2D  con la clase como Z
import matplotlib.pyplot as plt
from matplotlib import style

def dibuja_grafica2D(i,j,k):

    style.use('ggplot')
    
    x = df_tumoral.eval(df_tumoral.columns[i])
    y =  df_tumoral.eval(df_tumoral.columns[j])

    x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[i])
    y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[j])
    
    fig = plt.figure() 

    # AÑADIMOS UN SUBPLOT sobre fig
    ax1 = fig.add_subplot(111)

    ax1.scatter(x, y, c='g', marker='o')
    ax1.scatter(x2, y2, c ='r', marker='o')

    plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
    plt.scatter(x2,y2, label='SANAS', color='r', s=25, marker="o")

    # ax1.scatter(x, y, z, c='g', marker='o')
    # ax1.scatter(x2, y2, z2, c ='r', marker='o')

    ax1.set_xlabel(df_tumoral.columns[i])
    ax1.set_ylabel(df_tumoral.columns[j])


    plt.title('Breast Cancer Dataset Analysis - Human Forecast')
    plt.legend()
    #plt.show()
    plt.savefig('graficos/graf_'+df_tumoral.columns[i]+df_tumoral.columns[j]+'.png')


dibuja_grafica2D(1,2,9)

In [ ]:
# pero ahora solo estamos interesados en combinar dos a dos 
# con z que sea la clase de tumor que es el indice 9

for counter1 in range(0,len(df.columns)-2):
    for counter2 in range(counter1 + 1,len(df.columns)-1):
            try:
                dibuja_grafica2D(counter1,counter2,9)
                #print counter1,",", counter2
            except Exception:
                pass